In [2]:
import networkx as nx
import plotly.graph_objects as go

# ==========================================
# 1. Setup the Graph and Layout
# ==========================================
G = nx.Graph()
edges = [
    (0, 1), (1, 3), (1, 4),  # Left branch
    (0, 2), (2, 5), (2, 6)   # Right branch
]
G.add_edges_from(edges)

pos = nx.spring_layout(G, seed=42)

def get_edge_coords(edge_list):
    x, y = [], []
    for u, v in edge_list:
        x += [pos[u][0], pos[v][0], None]
        y += [pos[u][1], pos[v][1], None]
    return x, y

# Fixed free variable for the animation
FREE_VAR_NODE = 0

# ==========================================
# 2. Define the Recursive Decomposition Tree
# ==========================================
decomposition_tree = {
    "name": "Root",
    "edges": edges,
    "children": [
        {
            "name": "Left Subgraph",
            "edges": [(0, 1), (1, 3), (1, 4)],
            "children": [
                {"name": "Edge (0-1)", "edges": [(0, 1)], "children": []},
                {
                    "name": "Sub-Component (Node 1)",
                    "edges": [(1, 3), (1, 4)],
                    "children": [
                        {"name": "Edge (1-3)", "edges": [(1, 3)], "children": []},
                        {"name": "Edge (1-4)", "edges": [(1, 4)], "children": []}
                    ]
                }
            ]
        },
        {
            "name": "Right Subgraph",
            "edges": [(0, 2), (2, 5), (2, 6)],
            "children": [
                {"name": "Edge (0-2)", "edges": [(0, 2)], "children": []},
                {
                    "name": "Sub-Component (Node 2)",
                    "edges": [(2, 5), (2, 6)],
                    "children": [
                        {"name": "Edge (2-5)", "edges": [(2, 5)], "children": []},
                        {"name": "Edge (2-6)", "edges": [(2, 6)], "children": []}
                    ]
                }
            ]
        }
    ]
}

# ==========================================
# 3. Generate Animation States
# ==========================================
history = []
global_solved = set()

def build_history(node, depth=0):
    current_scope = set(node["edges"])
    
    # Step 1: Recurse Down
    history.append({
        "focus": current_scope,
        "solved": global_solved.copy(),
        "color_focus": "#FFD700", # Yellow
    })
    
    # Step 2: Process Children
    if not node["children"]:
        # Base Case
        history.append({
            "focus": current_scope,
            "solved": global_solved.copy(),
            "color_focus": "#FF4500", # Orange Red
        })
        global_solved.update(current_scope)
    else:
        for child in node["children"]:
            build_history(child, depth + 1)
            
    # Step 3: Construct Up
    history.append({
        "focus": set(), 
        "solved": global_solved.copy(), 
        "color_focus": "white",
    })

build_history(decomposition_tree)

# ==========================================
# 4. Create Visual Elements
# ==========================================

# Base edges (static background)
bx, by = get_edge_coords(edges)
base_edge_trace = go.Scatter(
    x=bx, y=by,
    mode='lines',
    line=dict(width=2, color='#d3d3d3'), # Light gray
    hoverinfo='none',
    showlegend=False
)

def make_frame(step_data):
    traces = [base_edge_trace]
    
    # 1. Solved Edges (Constructed Map)
    if step_data["solved"]:
        sx, sy = get_edge_coords(list(step_data["solved"]))
        solved_trace = go.Scatter(
            x=sx, y=sy, mode='lines',
            line=dict(width=6, color='#00BFFF'), # Deep Sky Blue
            name='Constructed',
            hoverinfo='none'
        )
        traces.append(solved_trace)

    # 2. Focus Edges
    focus_edges = step_data["focus"]
    if focus_edges:
        fx, fy = get_edge_coords(list(focus_edges))
        focus_trace = go.Scatter(
            x=fx, y=fy, mode='lines',
            line=dict(width=6, color=step_data["color_focus"]),
            name='Focus',
            hoverinfo='none'
        )
        traces.append(focus_trace)
        
    # 3. Nodes (Dynamic Coloring - Simplified)
    node_colors = []
    node_text = []
    
    for n in G.nodes():
        if n == FREE_VAR_NODE:
            node_colors.append("#9370DB") # Purple for Free Var
            node_text.append("Free Var")
        else:
            node_colors.append("#808080") # Grey
            node_text.append("")

    node_x = [pos[n][0] for n in G.nodes()]
    node_y = [pos[n][1] for n in G.nodes()]
    
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="top center",
        textfont=dict(size=14, color="black"), # Black text
        marker=dict(
            size=25, 
            color=node_colors, 
            line=dict(width=2, color='black')
        ),
        hoverinfo='none'
    )
    traces.append(node_trace)
    
    return traces

# ==========================================
# 5. Build Animation
# ==========================================
frames = []
for i, step in enumerate(history):
    data_objs = make_frame(step)
    frames.append(go.Frame(
        data=data_objs,
        name=str(i),
        # Title update removed
    ))

init_data = make_frame(history[0])

fig = go.Figure(
    data=init_data,
    frames=frames,
    layout=go.Layout(
        # Title removed
        template="plotly_white", # White background
        showlegend=False,
        xaxis=dict(showgrid=False, zeroline=False, visible=False),
        yaxis=dict(showgrid=False, zeroline=False, visible=False),
        height=600,
        margin=dict(t=30, b=30, l=30, r=30), # Adjusted margins
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {
                    "label": "▶ Play",
                    "method": "animate",
                    "args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]
                },
                {
                    "label": "⏸ Pause",
                    "method": "animate",
                    "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]
                }
            ],
            "showactive": False,
            "y": -0.1, "x": 0.5, "xanchor": "center"
        }]
    )
)

# Slider
steps = []
for i in range(len(history)):
    steps.append({
        "method": "animate",
        "label": str(i),
        "args": [[str(i)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}]
    })

fig.update_layout(
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {"prefix": "Step: "},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.05, "y": -0.2,
        "steps": steps
    }]
)

fig.show()